In [25]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle

In [26]:
# from google.colab import drive
# drive.mount('/content/drive')

path = '../Resources/loan.csv'

In [27]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
# file_path = Path(f'{path}loan.csv')
loan_data = pd.read_csv(path)

# Review the DataFrame
loan_data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [28]:
# Drop Loan_ID column
columns_to_delete = ['Loan_ID']
loan_data = loan_data.drop(columns = columns_to_delete)

# Separately transforming the Loan_Status column without using OHE, since it needs to remain in one copy
loan_data['Loan_Status'] = loan_data['Loan_Status'].map({'N': 0, 'Y': 1})

# Split to train and test. Splitting before knn imputing required for fitting imputer on train and fill NaNs using it on test
df_train, df_test = train_test_split(loan_data, test_size=0.2, random_state=1)

# get categorical and numerical columns
categorical_columns = loan_data.select_dtypes(include=['object', 'category']).columns
numerical_columns = loan_data.select_dtypes(include=['int64', 'float64']).columns

In [29]:
#Impute missing values for continuous data only (LoanAmount, Loan_Amount_Term)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Apply the KNN imputer to the numerical columns
df_train[numerical_columns] = imputer.fit_transform(df_train[numerical_columns])

# apply to test !IMPORTANT! for prevent data leak
df_test[numerical_columns] = imputer.transform(df_test[numerical_columns])

In [36]:
# Change all categorical columns to binary using get dummies
clean_train_df = pd.get_dummies(df_train, columns=['Gender','Married','Dependents','Education',
                                                    'Self_Employed','Property_Area'])
clean_test_df = pd.get_dummies(df_test, columns=['Gender','Married','Dependents','Education',
                                                    'Self_Employed','Property_Area'])
# Export columns to be used in app
columns = clean_train_df.columns.tolist()

# Save the column names to a CSV file
with open('columns_list.csv', 'w') as f:
    for column in columns:
        f.write(f"{column}\n")

In [31]:
# Separate the data into labels and features
# Separate the y variable, the labels
y_train = clean_train_df['Loan_Status']
y_test = clean_test_df['Loan_Status']

# Separate the X variable, the features
X_train = clean_train_df.drop('Loan_Status', axis=1)
X_test = clean_test_df.drop('Loan_Status', axis=1)
X_train

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
291,4400.0,0.0,127.0,360.0,0.0,False,True,False,True,False,False,True,False,True,False,True,False,False,True,False
507,3583.0,0.0,96.0,360.0,1.0,False,False,True,False,True,False,False,False,True,False,True,False,False,False,True
328,4333.0,2451.0,110.0,360.0,1.0,True,False,False,True,True,False,False,False,True,False,True,False,False,False,True
609,2900.0,0.0,71.0,360.0,1.0,True,False,True,False,True,False,False,False,True,False,True,False,True,False,False
69,4300.0,0.0,136.0,360.0,0.0,True,False,True,False,True,False,False,False,True,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,6080.0,2569.0,182.0,360.0,1.0,False,True,False,True,True,False,False,False,True,False,True,False,True,False,False
144,11757.0,0.0,187.0,180.0,1.0,False,True,False,True,False,False,True,False,True,False,True,False,False,False,True
72,3500.0,0.0,81.0,300.0,1.0,False,True,True,False,True,False,False,False,True,False,True,False,False,True,False
235,5500.0,1260.0,170.0,360.0,1.0,False,True,False,True,False,True,False,False,True,False,True,False,True,False,False


In [32]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [33]:
# Creating the neural network model
# The first dense layer with 64 neurons and ReLU (Rectified Linear Unit) activation function:
# The ReLU function is often used in hidden layers because it helps the model learn nonlinear relationships in the data and prevents the vanishing gradient problem.
# Dropout layer with a rate of 0.5: Dropout is used to prevent overfitting by randomly disabling neurons during training.
# A rate of 0.5 means that 50% of the neurons will be disabled at each training iteration.
# The second dense layer with 32 neurons and ReLU activation function: This layer helps the model learn more complex patterns in the data.
# Another Dropout layer with a rate of 0.5: For additional overfitting prevention.
# Output layer with 1 neuron and sigmoid activation function: Since the task is binary classification (Loan_Status - approved or not),
# the output layer with a sigmoid activation function returns a probability value (from 0 to 1), which is easily interpreted as the probability of the positive class.

neural_network_model = Sequential()
neural_network_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
neural_network_model.add(Dropout(0.5))
neural_network_model.add(Dense(32, activation='relu'))
neural_network_model.add(Dropout(0.5))
neural_network_model.add(Dense(1, activation='sigmoid'))

# Compiling the model
neural_network_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
neural_network_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluating the model on test data
loss, accuracy = neural_network_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

Epoch 1/50


/opt/anaconda3/envs/dev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5962 - loss: 0.7029 - val_accuracy: 0.7172 - val_loss: 0.5966
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5954 - loss: 0.6721 - val_accuracy: 0.7374 - val_loss: 0.5746
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6330 - loss: 0.6413 - val_accuracy: 0.7576 - val_loss: 0.5594
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6535 - loss: 0.6307 - val_accuracy: 0.7475 - val_loss: 0.5514
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6886 - loss: 0.6534 - val_accuracy: 0.7475 - val_loss: 0.5441
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6615 - loss: 0.6278 - val_accuracy: 0.7677 - val_loss: 0.5384
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7262 - loss: 0.6040 - val_accuracy: 0.7677 - val_loss: 0.5351
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7389 - loss: 0.5919 - val_accuracy: 0.7677 - val_loss: 0.5294
Ep

In [34]:
# Save the entire model (architecture + weights + optimizer state)
neural_network_model.save('model.h5')

# Save model architecture to JSON
model_architecture = neural_network_model.to_json()
with open('model_architecture.json', 'w') as f:
    f.write(model_architecture)

# Save model weights
neural_network_model.save_weights('model.weights.h5')


In [35]:
# Save choices for form inputs
choices = {
    'Combined_income': None,
    'Credit_history': ['Yes', 'No'],
    'Requested_amount': None,
    'Community_type': ['Urban', 'Rural', 'Suburban'],
    'Dependents': ['0', '1', '2', '3+'],
    'College_degree': ['Yes', 'No']
}

with open('choices.pkl', 'wb') as f:
    pickle.dump(choices, f)